In [1]:
import os
os.getcwd()
os.chdir('c:\\Users\\larry\\Desktop\\Geoguessr ML Proj\\Geolocation-Project\\reverse_geocode3')

In [2]:
import data_loader
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
from shapely.ops import nearest_points
from unidecode import unidecode
import numpy as np
import hdbscan
import re 
import hashlib


In [ ]:
include_all_worldcities = False
should_merge

In [4]:
def clean_city_data(df):
    """
    Cleans the city_name and alternatenames columns of a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame with 'city_name' and 'alternatenames' columns.

    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    # Create a copy to avoid SettingWithCopyWarning
    df = df.copy()    

    # 1. Fill any NaN values in 'alternatenames' with an empty string.
    # 2. Apply a lambda function to split the string by commas.
    # 3. Inside the lambda, use a list comprehension to strip whitespace from each resulting string.
    # 4. This ensures every entry in the 'alternatenames' column is a list of clean strings.
    df['alternatenames'] = df['alternatenames'].fillna('').apply(
        lambda x: [s.strip() for s in x.split(',') if s.strip()]
    )

    # Function to process each city name
    def process_city_name(row):
        # Use str() as a safeguard against potential non-string city names
        city_name = str(row['city_name'])

        alternatenames = list(row['alternatenames'])  # It's a list, so proceed

        # Always append the original city name to alternatenames
        # This preserves the original data before any modifications.
        if city_name:
             alternatenames.append(city_name.strip())

        # If country is PH, remove all traces of "Brgy." ---
        if row['city_country_code'] == 'PH':
            city_name = re.sub(r'Brgy\.\s*', '', city_name, flags=re.IGNORECASE).strip()

        # Extract content from square brackets
        bracketed_content = re.findall(r'\[(.*?)\]', city_name)
        if bracketed_content:
            for content in bracketed_content:
                alternatenames.extend([name.strip() for name in content.split(',')])
            city_name = re.sub(r'\s*\[.*?\]\s*', '', city_name).strip()

        # Extract content from parentheses
        parenthetical_content = re.findall(r'\((.*?)\)', city_name)
        if parenthetical_content:
            for content in parenthetical_content:
                alternatenames.extend([name.strip() for name in content.split(',')])
            city_name = re.sub(r'\s*\(.*?\)\s*', '', city_name).strip()

        # Split by slash and update city_name and alternatenames
        if '/' in city_name:
            parts = [part.strip() for part in city_name.split('/')]
            city_name = parts[-1]
            alternatenames.extend(parts[:-1])

        # Process commas
        if ',' in city_name:
            parts = [part.strip() for part in city_name.split(',')]
            city_name = parts[0]
            alternatenames.extend(parts[1:])

        # Clean up lists and remove duplicates
        row['city_name'] = city_name.strip()
        # Filter out any potential empty strings that may have been created during the process
        row['alternatenames'] = list(set(alt for alt in alternatenames if alt))
        return row

    # Apply the function to each row of the DataFrame
    df = df.apply(process_city_name, axis=1)
    return df


df, _ = data_loader.load_cities_and_build_kdtree_from_geonames()
df = clean_city_data(df) 
df["source"] = "geonames"

print(len(df))
df.head()

Loading city data from GeoNames cities500.txt...
Reading natural_earth_data/cities500.txt ...
Loaded 215735 entries from GeoNames.

--- Loading Admin Name Mappings ---
Loading Admin1 names from natural_earth_data/admin1CodesASCII.txt...
Loaded 3893 Admin1 name mappings.
Loading Admin2 names from natural_earth_data/admin2Codes.txt...
Loaded 47356 Admin2 name mappings.

--- Mapping Admin Codes to Names ---
Admin1 names mapped.
Admin2 names mapped.
Converting GeoNames data to GeoDataFrame...
Building KDTree for GeoNames cities...
KDTree built successfully.
215735


,city_name,city_latitude,city_longitude,city_country_code,city_population,alternatenames,city_admin1_name,city_admin2_name,geometry,source
0,Vila,42.53176,1.56654,AD,1418,"[Casas Vila, Vila]",Encamp,,POINT (1.56654 42.53176),geonames
1,Soldeu,42.57688,1.66769,AD,602,"[surudeu, سولدو, スルデウ, swldw, סולדאו, Sol'deu,...",Canillo,,POINT (1.66769 42.57688),geonames
2,Sispony,42.53368,1.51613,AD,833,[Sispony],La Massana,,POINT (1.51613 42.53368),geonames
3,El Tarter,42.57952,1.65362,AD,1052,"[Ehl Tarter, Эл Тартер, Ел Тартер, ال تارتر, E...",Canillo,,POINT (1.65362 42.57952),geonames
4,Sant Julià de Lòria,42.46372,1.49129,AD,8022,"[sheng hu li ya-de luo li ya, Sant-Zhulija-de-...",Sant Julià de Loria,,POINT (1.49129 42.46372),geonames


In [5]:
df[df["city_name"].str.contains(",")]

,city_name,city_latitude,city_longitude,city_country_code,city_population,alternatenames,city_admin1_name,city_admin2_name,geometry,source


In [6]:
def clean_city_data2(df):
    """
    Cleans the city_name and alternatenames columns of a DataFrame.

    Args:
        df (pd.DataFrame): The input DataFrame with 'city_name' and 'alternatenames' columns.

    Returns:
        pd.DataFrame: The cleaned DataFrame.
    """
    # Create a copy to avoid SettingWithCopyWarning
    df = df.copy()    

    def create_list_column(row, col1, col2):
        """
        Creates a list from two columns, excluding null values.

        Args:
            row: A row of the DataFrame.
            col1: The name of the first column.
            col2: The name of the second column.

        Returns:
            A list containing non-null values from col1 and col2.
        """
        values = []

        # **CRITICAL**: Only add the name if it's a non-empty string.
        # This prevents NaN (float) values from being added to the set.
        if isinstance(row[col1], str) and row[col1]:
            values.append(row[col1])
        if isinstance(row[col2], str) and row[col2]:
            values.append(row[col2])
        return values

    df['alternatenames'] = df.apply(lambda row: create_list_column(row, 'city', 'city_ascii'), axis=1)


    # Function to process each city name
    def process_city_name(row):
        # Use str() as a safeguard against potential non-string city names
        city_name = str(row['city_ascii'])

        alternatenames = list(row['alternatenames'])  # It's a list, so proceed

        # Always append the original city name to alternatenames
        # This preserves the original data before any modifications.
        if city_name:
             alternatenames.append(city_name.strip())

        # If country is PH, remove all traces of "Brgy." ---
        if row['iso2'] == 'PH':
            city_name = re.sub(r'Brgy\.\s*', '', city_name, flags=re.IGNORECASE).strip()

        # Extract content from square brackets
        bracketed_content = re.findall(r'\[(.*?)\]', city_name)
        if bracketed_content:
            for content in bracketed_content:
                alternatenames.extend([name.strip() for name in content.split(',')])
            city_name = re.sub(r'\s*\[.*?\]\s*', '', city_name).strip()

        # Extract content from parentheses
        parenthetical_content = re.findall(r'\((.*?)\)', city_name)
        if parenthetical_content:
            for content in parenthetical_content:
                alternatenames.extend([name.strip() for name in content.split(',')])
            city_name = re.sub(r'\s*\(.*?\)\s*', '', city_name).strip()

        # Split by slash and update city_name and alternatenames
        if '/' in city_name:
            parts = [part.strip() for part in city_name.split('/')]
            city_name = parts[-1]
            alternatenames.extend(parts[:-1])

        # Process commas
        if ',' in city_name:
            parts = [part.strip() for part in city_name.split(',')]
            city_name = parts[0]
            alternatenames.extend(parts[1:])

        # Clean up lists and remove duplicates
        row['city_ascii'] = city_name.strip()
        row["city"] = row["city"].strip()
        # Filter out any potential empty strings that may have been created during the process
        row['alternatenames'] = list(set(alt for alt in alternatenames if alt))
        return row

    # Apply the function to each row of the DataFrame
    df = df.apply(process_city_name, axis=1)
    return df


df2 = pd.read_csv('natural_earth_data/worldcities.csv')
if not include_all_worldcities:
    df2 = df2[df2["population"] <= 500]
df2['population'] = df2['population'].fillna(0)
nyc_filter = (df2['city'] == 'New York') & (df2['admin_name'] == 'New York')
df2.loc[nyc_filter, ['city', 'city_ascii']] = 'New York City'
print(len(df2))
df2["source"] = "worldcities"
df2 = clean_city_data2(df2)
df2.head()

219


,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id,source,alternatenames
47589,Tchitado,Tchitado,-17.3167,13.9167,Angola,AO,AGO,Cunene,NaN,500.0,1024158837,worldcities,[Tchitado]
47590,Yakossi,Yakossi,5.6170,23.3167,Central African Republic,CF,CAF,Mbomou,NaN,500.0,1140246753,worldcities,[Yakossi]
47591,Tmassah,Tmassah,26.3667,15.8000,Libya,LY,LBY,Murzuq,NaN,500.0,1434333715,worldcities,[Tmassah]
47592,Puerto Pinasco,Puerto Pinasco,-22.6400,-57.7900,Paraguay,PY,PRY,Presidente Hayes,NaN,500.0,1600670025,worldcities,[Puerto Pinasco]
47593,Oymyakon,Oymyakon,63.4608,142.7858,Russia,RU,RUS,Sakha (Yakutiya),NaN,500.0,1643797797,worldcities,[Oymyakon]


In [7]:
df2.tail()

,city,city_ascii,lat,lng,country,iso2,iso3,admin_name,capital,population,id,source,alternatenames
47803,Kostel,Kostel,45.5088,14.9100,Slovenia,SI,SVN,Kostel,admin,10.0,1705541759,worldcities,[Kostel]
47804,Kingoonya,Kingoonya,-30.9170,135.3147,Australia,AU,AUS,South Australia,NaN,4.0,1036942792,worldcities,[Kingoonya]
47826,Chuquicamata,Chuquicamata,-22.3169,-68.9301,Chile,CL,CHL,Antofagasta,NaN,0.0,1152468996,worldcities,[Chuquicamata]
47969,Nordvik,Nordvik,73.9975,111.4633,Russia,RU,RUS,Krasnoyarskiy Kray,NaN,0.0,1643587468,worldcities,[Nordvik]
47970,Logashkino,Logashkino,70.8500,153.9167,Russia,RU,RUS,Sakha (Yakutiya),NaN,0.0,1643050775,worldcities,[Logashkino]


In [8]:
from tqdm import tqdm

def normalize_text(text):
    """Converts a string to its closest ASCII representation and lowercases it."""
    if isinstance(text, str):
        return unidecode(text.lower())
    return text

def merge_geonames_with_worldcities(gdf_geonames, df_worldcities):
    """
    Intelligently merges worldcities data into a geonames GeoDataFrame,
    capturing alternate admin names and assigning a unique city_id.
    """
    print("--- Starting Intelligent Merge Process ---")
    
    # Add a column to store potential alternate admin names from worldcities
    gdf_geonames['worldcities_admin1_name'] = None

    # --- Part A: Create a fast lookup map from all geonames aliases ---
    print("Step A: Creating a lookup map from geonames names and aliases...")
    # ... (rest of Part A is unchanged) ...
    name_to_geonames_index = {}
    for idx, row in gdf_geonames.iterrows():
        country_code = row['city_country_code']
        admin_1_code = normalize_text(row['city_admin1_name'])
        main_name_norm = normalize_text(row['city_name'])
        if main_name_norm:
            name_to_geonames_index[(main_name_norm, admin_1_code, country_code)] = idx
        for alt_name in row['alternatenames']:
            alt_name_norm = normalize_text(alt_name)
            if alt_name_norm:
                name_to_geonames_index[(alt_name_norm, admin_1_code, country_code)] = idx
    print(f"  Created a map with {len(name_to_geonames_index)} unique name aliases.")

    # --- Part B: Iterate through worldcities and merge/append ---
    print("Step B: Matching worldcities against geonames map...")
    df_worldcities_processed = df_worldcities.copy()
    df_worldcities_processed['merged_into_geonames'] = False

    for idx, row in tqdm(df_worldcities_processed.iterrows(), total=df_worldcities_processed.shape[0], desc="  Processing worldcities"):
        country_code = row['iso2'] 
        admin_1_code = normalize_text(row['admin_name'])
        city_norm = normalize_text(row['city'])
        city_ascii_norm = normalize_text(row['city_ascii'])
        geonames_match_idx = name_to_geonames_index.get((city_norm, admin_1_code, country_code)) or name_to_geonames_index.get((city_ascii_norm, admin_1_code, country_code))

        if geonames_match_idx is not None:
            # Match found! Update alternatenames
            existing_alternames = gdf_geonames.at[geonames_match_idx, 'alternatenames']
            updated_alternames = set(existing_alternames)
            updated_alternames.update(row["alternatenames"])
            gdf_geonames.at[geonames_match_idx, 'alternatenames'] = sorted(list(updated_alternames))
            
            # *** NEW: Capture the admin name from worldcities as a potential alternate ***
            gdf_geonames.at[geonames_match_idx, 'worldcities_admin1_name'] = row['admin_name']

            df_worldcities_processed.at[idx, 'merged_into_geonames'] = True

    # --- Part C: Prepare and concatenate the non-merged cities ---
    print("Step C: Preparing and appending new cities from worldcities...")
    # ... (rest of Part C is mostly unchanged) ...
    new_cities = df_worldcities_processed[~df_worldcities_processed['merged_into_geonames']].copy()
    print(f"  Found {len(new_cities)} new cities to append.")

    if not new_cities.empty:
        new_cities.rename(columns={
            'city': 'city_name', 'lat': 'city_latitude', 'lng': 'city_longitude',
            'iso2': 'city_country_code', 'population': 'city_population',
            'admin_name': 'city_admin1_name'
        }, inplace=True)
        # For new cities, the worldcities admin name is just its own admin name
        new_cities['worldcities_admin1_name'] = new_cities['city_admin1_name']
        new_cities['city_admin2_name'] = ""
        new_cities['geometry'] = gpd.points_from_xy(new_cities['city_longitude'], new_cities['city_latitude'])
        
        final_columns = [col for col in gdf_geonames.columns if col in new_cities.columns]
        new_cities_gdf = gpd.GeoDataFrame(new_cities[final_columns], crs="EPSG:4326")
        
        gdf_merged = pd.concat([gdf_geonames, new_cities_gdf], ignore_index=True)
    else:
        gdf_merged = gdf_geonames.copy()

    # --- Part D: Finalize with unique ID and clean data types ---
    def create_hashed_id(row, cols_to_hash, length=16):
        """
        Creates a truncated SHA-256 hash from specified columns of a DataFrame row.
        """
        # Concatenate values with a separator for stability
        string_to_hash = ''.join([str(row[col]) for col in cols_to_hash])
        
        # Encode the string to bytes, required for hashlib
        encoded_string = string_to_hash.encode('utf-8')
        
        # Create the hash and return the first `length` characters of the hex digest
        sha256_hash = hashlib.sha256(encoded_string).hexdigest()
        
        return sha256_hash[:length]
    gdf_merged.insert(0, 'city_id', gdf_merged.index)
    gdf_merged["city_id"] = gdf_merged.apply(
        create_hashed_id, 
        cols_to_hash=["city_name", "city_latitude", "city_longitude", "city_country_code", "city_population", "city_admin1_name", "worldcities_admin1_name"], 
        axis=1
    )

    gdf_merged['city_population'] = pd.to_numeric(gdf_merged['city_population'], errors='coerce').astype(pd.Int64Dtype())


    print(f"\n--- Intelligent Merge Complete! {len(gdf_merged)} entries---")
    return gdf_merged

merged_gdf = merge_geonames_with_worldcities(df, df2)
merged_gdf = merged_gdf.drop_duplicates(subset=['city_id'], keep="first")

merged_gdf.head()

--- Starting Intelligent Merge Process ---
Step A: Creating a lookup map from geonames names and aliases...
  Created a map with 853439 unique name aliases.
Step B: Matching worldcities against geonames map...


  Processing worldcities: 100%|██████████| 219/219 [00:00<00:00, 12115.55it/s]


Step C: Preparing and appending new cities from worldcities...
  Found 145 new cities to append.

--- Intelligent Merge Complete! 215880 entries---


,city_id,city_name,city_latitude,city_longitude,city_country_code,city_population,alternatenames,city_admin1_name,city_admin2_name,geometry,source,worldcities_admin1_name
0,48e8d39f793285b5,Vila,42.53176,1.56654,AD,1418,"[Casas Vila, Vila]",Encamp,,POINT (1.56654 42.53176),geonames,None
1,a5f18e80ec393424,Soldeu,42.57688,1.66769,AD,602,"[surudeu, سولدو, スルデウ, swldw, סולדאו, Sol'deu,...",Canillo,,POINT (1.66769 42.57688),geonames,None
2,38132557d06759a4,Sispony,42.53368,1.51613,AD,833,[Sispony],La Massana,,POINT (1.51613 42.53368),geonames,None
3,6223f9846326a714,El Tarter,42.57952,1.65362,AD,1052,"[Ehl Tarter, Эл Тартер, Ел Тартер, ال تارتر, E...",Canillo,,POINT (1.65362 42.57952),geonames,None
4,8c4de62071e99deb,Sant Julià de Lòria,42.46372,1.49129,AD,8022,"[sheng hu li ya-de luo li ya, Sant-Zhulija-de-...",Sant Julià de Loria,,POINT (1.49129 42.46372),geonames,None


In [9]:
print(len(merged_gdf))
print(len(set(merged_gdf["city_id"])))

215880
215880


In [10]:
import hdbscan
import numpy as np

def cluster_cities_with_hdbscan_three_tier_simplified(
    gdf_all_cities,
    min_cluster_size=5,
    min_samples=None,
    cluster_selection_epsilon_meters=None
):
    """
    Identifies a three-tiered metropolitan hierarchy using HDBSCAN on a pre-merged dataset.
    This version uses a persistent `city_id` for all associations.
    """
    print("--- Starting Three-Tier HDBSCAN Clustering Process ---")

    # --- Step 1: Prepare Data ---
    print("Step 1: Preparing combined data for clustering...")
    gdf_proc = gdf_all_cities.copy()
    
    # Normalize admin names and fill NaNs
    for col in ['city_admin1_name', 'city_admin2_name']:
        gdf_proc[col] = gdf_proc[col].apply(normalize_text).fillna('')
    print(f"  Processing {len(gdf_proc)} total cities.")
    
    # --- Step 2: Project Data and Run HDBSCAN ---
    print("\nStep 2: Running HDBSCAN clustering...")
    gdf_proj = gdf_proc.to_crs(epsg=3857)
    coords = np.array(list(zip(gdf_proj.geometry.x, gdf_proj.geometry.y)))
    clusterer = hdbscan.HDBSCAN(
        min_cluster_size=min_cluster_size, min_samples=min_samples, metric='euclidean',
        cluster_selection_epsilon=cluster_selection_epsilon_meters or 0
    )
    clusterer.fit(coords)
    gdf_proc['cluster_id'] = clusterer.labels_
    print(f"  HDBSCAN found {len(np.unique(clusterer.labels_)) - 1} clusters.")

    # --- Step 3: Identify Representatives for All Three Tiers ---
    print("\nStep 3: Identifying representatives for all three metro tiers...")
    clustered_cities = gdf_proc[gdf_proc['cluster_id'] != -1]

    # Find the index of the most populous city for each tier
    idx_greater = clustered_cities.groupby('cluster_id')['city_population'].idxmax()
    idx_greater_country = clustered_cities.groupby(['cluster_id', 'city_country_code'])['city_population'].idxmax()
    idx_local = clustered_cities.groupby(['cluster_id', 'city_country_code', 'city_admin1_name', 'city_admin2_name'])['city_population'].idxmax()

    # Create mapping tables using the city_id of the representative cities
    greater_map = gdf_proc.loc[idx_greater, ['cluster_id', 'city_name', 'city_id']].rename(
        columns={'city_name': 'greater_metro_name', 'city_id': 'greater_metro_id'}
    )
    greater_country_map = gdf_proc.loc[idx_greater_country, ['cluster_id', 'city_country_code', 'city_name', 'city_id']].rename(
        columns={'city_name': 'greater_metro_in_country_name', 'city_id': 'greater_metro_in_country_id'}
    )
    local_map = gdf_proc.loc[idx_local, ['cluster_id', 'city_country_code', 'city_admin1_name', 'city_admin2_name', 'city_name', 'city_id']].rename(
        columns={'city_name': 'metro_name', 'city_id': 'metro_id'}
    )

    # --- Step 4: Map All Tiers Back to the Main DataFrame ---
    print("\nStep 4: Merging three-tier results back into the DataFrame...")
    gdf_merged = gdf_proc.merge(greater_map, on='cluster_id', how='left')
    gdf_merged = gdf_merged.merge(greater_country_map, on=['cluster_id', 'city_country_code'], how='left')
    gdf_merged = gdf_merged.merge(local_map, on=['cluster_id', 'city_country_code', 'city_admin1_name', 'city_admin2_name'], how='left')

    # For any city not in a cluster, it is its own metro representative
    for tier_prefix in ['metro', 'greater_metro_in_country', 'greater_metro']:
        is_unmatched = gdf_merged[f'{tier_prefix}_name'].isna()
        gdf_merged.loc[is_unmatched, f'{tier_prefix}_name'] = gdf_merged.loc[is_unmatched, 'city_name']
        gdf_merged.loc[is_unmatched, f'{tier_prefix}_id'] = gdf_merged.loc[is_unmatched, 'city_id']

    # --- Step 5: Final Cleanup ---
    # Normalize metro names and ensure ID columns are integer
    for col in ["greater_metro_name", "greater_metro_in_country_name", "metro_name"]:
        gdf_merged[col] = gdf_merged[col].apply(normalize_text).fillna('')
    for col in ["greater_metro_id", "greater_metro_in_country_id", "metro_id"]:
        gdf_merged[col] = gdf_merged[col].astype(str)

    # Add normalized city name and ensure alternames are unique
    gdf_merged.insert(loc=2, column='city_name_normalized', value=gdf_merged["city_name"].apply(normalize_text))
    def create_list_column(row, col1, col2):
        """
        Creates a list from two columns, excluding null values.

        Args:
            row: A row of the DataFrame.
            col1: The name of the first column.
            col2: The name of the second column.

        Returns:
            A list containing non-null values from col1 and col2.
        """
        values = row["alternatenames"]

        # **CRITICAL**: Only add the name if it's a non-empty string.
        # This prevents NaN (float) values from being added to the set.
        if isinstance(row[col1], str) and row[col1]:
            values.append(row[col1])
        if isinstance(row[col2], str) and row[col2]:
            values.append(row[col2])
        
        return list(set(values))

    gdf_merged['alternatenames'] = gdf_merged.apply(lambda row: create_list_column(row, 'city_name', 'city_name_normalized'), axis=1)
    gdf_merged['alternatenames'] = gdf_merged['alternatenames'].apply(lambda x: sorted(list(set(x))))
    
    # Define and order final columns
    final_cols = [
        'city_id', 'city_name', 'city_name_normalized', 'city_latitude', 'city_longitude',
        'city_country_code', 'city_population', 'alternatenames', 'city_admin1_name',
        'city_admin2_name', 'geometry', 'metro_name',
        'metro_id', 'greater_metro_name', 'greater_metro_id',
        'greater_metro_in_country_name', 'greater_metro_in_country_id',  'source'
    ]
    gdf_final = gdf_merged[final_cols]
    
    print("\n--- Process Complete! ---")
    return gdf_final

final_gdf_tuned = cluster_cities_with_hdbscan_three_tier_simplified(merged_gdf)

print(f"Final DataFrame has {len(final_gdf_tuned)} entries.")
final_gdf_tuned.head()

--- Starting Three-Tier HDBSCAN Clustering Process ---
Step 1: Preparing combined data for clustering...
  Processing 215880 total cities.

Step 2: Running HDBSCAN clustering...


c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\venv\Lib\site-packages\sklearn\utils\deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


  HDBSCAN found 5756 clusters.

Step 3: Identifying representatives for all three metro tiers...

Step 4: Merging three-tier results back into the DataFrame...

--- Process Complete! ---
Final DataFrame has 215880 entries.


,city_id,city_name,city_name_normalized,city_latitude,city_longitude,city_country_code,city_population,alternatenames,city_admin1_name,city_admin2_name,geometry,metro_name,metro_id,greater_metro_name,greater_metro_id,greater_metro_in_country_name,greater_metro_in_country_id,source
0,48e8d39f793285b5,Vila,vila,42.53176,1.56654,AD,1418,"[Casas Vila, Vila, vila]",encamp,,POINT (1.56654 42.53176),encamp,6176dbe2fec29734,andorra la vella,e098c520e9b6b97a,andorra la vella,e098c520e9b6b97a,geonames
1,a5f18e80ec393424,Soldeu,soldeu,42.57688,1.66769,AD,602,"[Sol'deu, Soldeu, soldeu, surudeu, swldw, Соль...",canillo,,POINT (1.66769 42.57688),canillo,f92dba31f62d7810,andorra la vella,e098c520e9b6b97a,andorra la vella,e098c520e9b6b97a,geonames
2,38132557d06759a4,Sispony,sispony,42.53368,1.51613,AD,833,"[Sispony, sispony]",la massana,,POINT (1.51613 42.53368),la massana,8334e238d8413912,andorra la vella,e098c520e9b6b97a,andorra la vella,e098c520e9b6b97a,geonames
3,6223f9846326a714,El Tarter,el tarter,42.57952,1.65362,AD,1052,"[Ehl Tarter, El Tarter, El Tarter - Principau ...",canillo,,POINT (1.65362 42.57952),canillo,f92dba31f62d7810,andorra la vella,e098c520e9b6b97a,andorra la vella,e098c520e9b6b97a,geonames
4,8c4de62071e99deb,Sant Julià de Lòria,sant julia de loria,42.46372,1.49129,AD,8022,"[San Julia, San Julià, Sant Julia de Loria, Sa...",sant julia de loria,,POINT (1.49129 42.46372),sant julia de loria,8c4de62071e99deb,andorra la vella,e098c520e9b6b97a,andorra la vella,e098c520e9b6b97a,geonames


In [11]:
def create_admin_mappings(gdf_in):
    """
    Creates mapping tables for admin1 and admin2 names with unique IDs, 
    alternate names, and country context. This version uses normalized names
    and country codes for reliable joining and uniqueness.
    
    Args:
        gdf_in (gpd.GeoDataFrame): The input GeoDataFrame which must contain:
            'city_admin1_name', 'city_admin2_name', 'city_country_code',
            'city_population', and 'worldcities_admin1_name'.

    Returns:
        tuple: A tuple containing two pandas DataFrames:
               (admin1_map_df, admin2_map_df).
    """
    print("--- Creating Admin Name Mapping Tables ---")
    gdf = gdf_in.copy()
    
    # Pre-calculate normalized names to be used as keys for grouping
    gdf['city_admin1_name_normalized'] = gdf['city_admin1_name'].apply(normalize_text)
    gdf['city_admin2_name_normalized'] = gdf['city_admin2_name'].apply(normalize_text)

    # --- Admin1 Mapping ---
    print("Processing Admin1 names...")
    admin1_data = []
    
    # Filter for valid admin1 entries and group by normalized name and country code
    admin1_gdf = gdf[gdf['city_admin1_name_normalized'].notna() & (gdf['city_admin1_name_normalized'] != '')].copy()
    grouped_admin1 = admin1_gdf.groupby(['city_admin1_name_normalized', 'city_country_code'])
    
    for (name_norm, country_code), group in grouped_admin1:
        # Choose the display name from the most populous city within the group
        rep_row = group.loc[group['city_population'].fillna(0).idxmax()]
        display_name = rep_row['city_admin1_name']
        
        # Collect all possible alternate names from the group
        alt_names = set()
        # Add all original admin1 names (e.g., "Tōkyō")
        alt_names.update(n for n in group['city_admin1_name'].dropna() if n.strip())
        # Add all worldcities admin1 names (e.g., "Tokyo")
        alt_names.update(n for n in group['worldcities_admin1_name'].dropna() if n.strip())
        
        # Add normalized versions of all collected names (e.g., "tokyo")
        normalized_alts = {normalize_text(name) for name in alt_names}
        alt_names.update(normalized_alts)
        
        admin1_data.append({
            'admin1_name_normalized': name_norm,
            'country_code': country_code,
            'admin1_name': display_name,
            'admin1_alternatenames': sorted(list(alt_names))
        })

    admin1_map_df = pd.DataFrame(admin1_data)
    # Sort for a stable index before assigning the ID
    admin1_map_df = admin1_map_df.sort_values(['country_code', 'admin1_name_normalized']).reset_index(drop=True)
    admin1_map_df.insert(0, 'admin1_id', admin1_map_df.index.astype(str))
    print(f"  Created Admin1 map with {len(admin1_map_df)} unique entries.")

    # --- Admin2 Mapping ---
    print("Processing Admin2 names...")
    
    # Merge the new admin1_id into the main dataframe to link admin2 to its parent admin1
    # **FIXED**: Use left_on/right_on because the column names are different.
    gdf = gdf.merge(
        admin1_map_df[['admin1_name_normalized', 'country_code', 'admin1_id']],
        left_on=['city_admin1_name_normalized', 'city_country_code'],
        right_on=['admin1_name_normalized', 'country_code'],
        how='left'
    )
    
    admin2_data = []
    # Filter for valid admin2 entries
    admin2_gdf = gdf[gdf['city_admin2_name_normalized'].notna() & (gdf['city_admin2_name_normalized'] != '')].copy()
    
    # Group by normalized name and its parent admin1_id for uniqueness
    grouped_admin2 = admin2_gdf.groupby(['city_admin2_name_normalized', 'admin1_id'])
    
    for (name_norm, admin1_id), group in grouped_admin2:
        if pd.isna(admin1_id):
            continue # Skip admin2 areas that couldn't be linked to a valid admin1
            
        # Choose display name and country code from the most populous city in the group
        rep_row = group.loc[group['city_population'].fillna(0).idxmax()]
        display_name = rep_row['city_admin2_name']
        country_code = rep_row['city_country_code']
        
        # Collect all possible alternate names
        alt_names = set()
        alt_names.update(n for n in group['city_admin2_name'].dropna() if n.strip())
        
        # Add normalized versions
        normalized_alts = {normalize_text(name) for name in alt_names}
        alt_names.update(normalized_alts)

        admin2_data.append({
            'admin2_name_normalized': name_norm,
            'admin2_name': display_name,
            'admin1_id': str(admin1_id), # Ensure consistent string type
            'country_code': country_code,
            'admin2_alternatenames': sorted(list(alt_names))
        })

    admin2_map_df = pd.DataFrame(admin2_data)
    # Sort for a stable index before assigning the ID
    admin2_map_df = admin2_map_df.sort_values(['country_code', 'admin1_id', 'admin2_name_normalized']).reset_index(drop=True)
    admin2_map_df.insert(0, 'admin2_id', admin2_map_df.index.astype(str))
    print(f"  Created Admin2 map with {len(admin2_map_df)} unique entries.")

    return admin1_map_df, admin2_map_df

In [12]:
import json
import os

# 2. Create the admin mapping tables
admin1_map, admin2_map = create_admin_mappings(merged_gdf)
display(admin1_map.head())
display(admin2_map.head())

# 3. Merge the new admin IDs into the final, clustered DataFrame
# **FIXED**: The merges now use compound keys for accuracy.
# Admin1 is unique by (name, country_code). Admin2 is unique by (name, admin1_id).

# Merge Admin1 ID
# Note: final_gdf_tuned['city_admin1_name'] is already normalized from the clustering step.
final_gdf_tuned = final_gdf_tuned.merge(
    admin1_map[['admin1_name_normalized', 'country_code', 'admin1_id']],
    left_on=['city_admin1_name', 'city_country_code'],
    right_on=['admin1_name_normalized', 'country_code'],
    how='left'
).rename(columns={'admin1_id': 'city_admin1_id'}).drop(columns=['admin1_name_normalized', 'country_code'])

final_gdf_tuned['city_admin1_id'] = final_gdf_tuned['city_admin1_id'].fillna('')

# Merge Admin2 ID
# Note: final_gdf_tuned['city_admin2_name'] is also normalized.
final_gdf_tuned = final_gdf_tuned.merge(
    admin2_map[['admin2_name_normalized', 'admin1_id', 'admin2_id']],
    left_on=['city_admin2_name', 'city_admin1_id'],
    right_on=['admin2_name_normalized', 'admin1_id'],
    how='left'
).rename(columns={'admin2_id': 'city_admin2_id'}).drop(columns=['admin2_name_normalized', 'admin1_id'])

# 4. Finalize data types and handle any remaining NaNs from the left merges
# The admin IDs are created as strings, so NaNs from the merge will be float `np.nan`.
# We fill them with an empty string for consistency.
final_gdf_tuned['city_admin1_id'] = final_gdf_tuned['city_admin1_id'].fillna('')
final_gdf_tuned['city_admin2_id'] = final_gdf_tuned['city_admin2_id'].fillna('')

# 5. Reorder columns to place IDs next to their names
final_cols_ordered = [
    'city_id', 'city_name', 'city_name_normalized', 'city_latitude', 'city_longitude',
    'city_country_code', 'city_population', 'alternatenames', 
    'city_admin1_name', 'city_admin1_id',
    'city_admin2_name', 'city_admin2_id', 
    'geometry', 'metro_name', 'metro_id',
    'greater_metro_name', 'greater_metro_id', 'greater_metro_in_country_name',
    'greater_metro_in_country_id',  'source'
]
final_gdf_tuned = final_gdf_tuned[[col for col in final_cols_ordered if col in final_gdf_tuned.columns]]

# 6. Display results to confirm the fix
print("\n--- Final GeoDataFrame with Admin IDs (as strings) ---")
display(final_gdf_tuned.head())


# 7. Export all files (optional, uncomment to run)
output_dir = 'c:\\Users\\larry\\Desktop\\Geoguessr ML Proj\\Geolocation-Project\\reverse_geocode3\\outputs'
os.makedirs(output_dir, exist_ok=True)

# Save the admin maps to CSV
if not include_all_worldcities:
    admin1_map.to_csv(os.path.join(output_dir, 'admin1_map_under500.csv'), index=False)
    admin2_map.to_csv(os.path.join(output_dir, 'admin2_map_under500.csv'), index=False)

else:
    admin1_map.to_csv(os.path.join(output_dir, 'admin1_map.csv'), index=False)
    admin2_map.to_csv(os.path.join(output_dir, 'admin2_map.csv'), index=False)
print(f"\nSuccessfully saved admin maps to '{output_dir}/'")

# Prepare and save the main GeoJSON
final_gdf_for_export = final_gdf_tuned.copy()
final_gdf_for_export['alternatenames'] = final_gdf_for_export['alternatenames'].apply(json.dumps)
geojson_path = os.path.join(output_dir, 'combined_cities.geojson')
if not include_all_worldcities:
    geojson_path = os.path.join(output_dir, 'combined_cities_under500.geojson')

final_gdf_for_export.to_file(geojson_path, driver='GeoJSON')
print(f"Successfully saved final data to '{geojson_path}'")

--- Creating Admin Name Mapping Tables ---
Processing Admin1 names...
  Created Admin1 map with 3826 unique entries.
Processing Admin2 names...
  Created Admin2 map with 29052 unique entries.


,admin1_id,admin1_name_normalized,country_code,admin1_name,admin1_alternatenames
0,0,andorra la vella,AD,Andorra la Vella,"[Andorra la Vella, andorra la vella]"
1,1,canillo,AD,Canillo,"[Canillo, canillo]"
2,2,encamp,AD,Encamp,"[Encamp, encamp]"
3,3,escaldes-engordany,AD,Escaldes-Engordany,"[Escaldes-Engordany, escaldes-engordany]"
4,4,la massana,AD,La Massana,"[La Massana, la massana]"


,admin2_id,admin2_name_normalized,admin2_name,admin1_id,country_code,admin2_alternatenames
0,0,abu dhabi municipality,Abu Dhabi Municipality,7,AE,"[Abu Dhabi Municipality, abu dhabi municipality]"
1,1,al ain municipality,Al Ain Municipality,7,AE,"[Al Ain Municipality, al ain municipality]"
2,2,al dhafra,Al Dhafra,7,AE,"[Al Dhafra, al dhafra]"
3,3,sector 1,Sector 1,9,AE,"[Sector 1, sector 1]"
4,4,sector 2,Sector 2,9,AE,"[Sector 2, sector 2]"



--- Final GeoDataFrame with Admin IDs (as strings) ---


,city_id,city_name,city_name_normalized,city_latitude,city_longitude,city_country_code,city_population,alternatenames,city_admin1_name,city_admin1_id,city_admin2_name,city_admin2_id,geometry,metro_name,metro_id,greater_metro_name,greater_metro_id,greater_metro_in_country_name,greater_metro_in_country_id,source
0,48e8d39f793285b5,Vila,vila,42.53176,1.56654,AD,1418,"[Casas Vila, Vila, vila]",encamp,2,,,POINT (1.56654 42.53176),encamp,6176dbe2fec29734,andorra la vella,e098c520e9b6b97a,andorra la vella,e098c520e9b6b97a,geonames
1,a5f18e80ec393424,Soldeu,soldeu,42.57688,1.66769,AD,602,"[Sol'deu, Soldeu, soldeu, surudeu, swldw, Соль...",canillo,1,,,POINT (1.66769 42.57688),canillo,f92dba31f62d7810,andorra la vella,e098c520e9b6b97a,andorra la vella,e098c520e9b6b97a,geonames
2,38132557d06759a4,Sispony,sispony,42.53368,1.51613,AD,833,"[Sispony, sispony]",la massana,4,,,POINT (1.51613 42.53368),la massana,8334e238d8413912,andorra la vella,e098c520e9b6b97a,andorra la vella,e098c520e9b6b97a,geonames
3,6223f9846326a714,El Tarter,el tarter,42.57952,1.65362,AD,1052,"[Ehl Tarter, El Tarter, El Tarter - Principau ...",canillo,1,,,POINT (1.65362 42.57952),canillo,f92dba31f62d7810,andorra la vella,e098c520e9b6b97a,andorra la vella,e098c520e9b6b97a,geonames
4,8c4de62071e99deb,Sant Julià de Lòria,sant julia de loria,42.46372,1.49129,AD,8022,"[San Julia, San Julià, Sant Julia de Loria, Sa...",sant julia de loria,6,,,POINT (1.49129 42.46372),sant julia de loria,8c4de62071e99deb,andorra la vella,e098c520e9b6b97a,andorra la vella,e098c520e9b6b97a,geonames



Successfully saved admin maps to 'c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\reverse_geocode3\outputs/'
Successfully saved final data to 'c:\Users\larry\Desktop\Geoguessr ML Proj\Geolocation-Project\reverse_geocode3\outputs\combined_cities_under500.geojson'


In [13]:
final_gdf_tuned[final_gdf_tuned["city_name"] == "Tokyo"]

,city_id,city_name,city_name_normalized,city_latitude,city_longitude,city_country_code,city_population,alternatenames,city_admin1_name,city_admin1_id,city_admin2_name,city_admin2_id,geometry,metro_name,metro_id,greater_metro_name,greater_metro_id,greater_metro_in_country_name,greater_metro_in_country_id,source
123737,3e89082cad630aec,Tokyo,tokyo,35.6895,139.69171,JP,9733276,"[Edo, TYO, Tochiu, Tocio, Tokija, Tokijas, Tok...",tokyo,1495,,,POINT (139.69171 35.6895),tokyo,3e89082cad630aec,tokyo,3e89082cad630aec,tokyo,3e89082cad630aec,geonames


In [14]:
merged_gdf[merged_gdf["city_name"] == "Tokyo"]

,city_id,city_name,city_latitude,city_longitude,city_country_code,city_population,alternatenames,city_admin1_name,city_admin2_name,geometry,source,worldcities_admin1_name
123737,3e89082cad630aec,Tokyo,35.6895,139.69171,JP,9733276,"[Tocio, Edo, Tokió, tokyo, dong jing, টোকিও, d...",Tokyo,,POINT (139.69171 35.6895),geonames,None


In [15]:
admin1_map[admin1_map["admin1_id"] == "2259"]

,admin1_id,admin1_name_normalized,country_code,admin1_name,admin1_alternatenames
2259,2259,manica,MZ,Manica,"[Manica, manica]"


In [16]:
len(final_gdf_tuned)

215880

Made direct changes to cities_500.txt:
Some parentheses didn't have useful information, so they were removed
downtown honolulu got changed with parentheses
cites called Ninguno in geonames had to have changes from google maps
worldcities NYC was explicitly renamed

Cluster logic was only tested on phoenix and NY

Should have 239360 cities on all combine
215880 cities on under 500 combine